# Символьное дифференцирование

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку, куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 18 ноября 10:00**

Символьное дифференцирование - это инструмент для автоматического вывода формул производных, который открывает возможности для анализа сложных функций, оптимизации процессов и работы с уравнениями. Мы уже на многих занятиях сталкивались с этой темой - давайте попробуем реализовать собственное!

## Выражение

Создадим основной класс `Expr`, от которого будут наследоваться различные типы выражений, такие как константы, переменные, суммы, произведения и другие. Класс должен содержать методы:
* `__call__`, который будет вычислять значение выражения, используя переданный ему контекст (словарь, связывающий имена переменных с их значениями).
* `d`, принимающий имя переменной, по которой требуется вычислить производную, и возвращающий выражение, представляющее производную по этой переменной.

Эти методы нужно будет переопределить в каждом из подклассов для корректного выполнения операций.

In [9]:
class Expr:
    def __call__(self, **context):
        pass

    def d(self, wrt):
        pass

    def __add__(self, other):
        return Sum(self, other)

    def __mul__(self, other):
        return Product(self, other)

    def __truediv__(self, other):
        return Fraction(self, other)

    def __sub__(self, other):
        return self + Const(-1) * other

    def __neg__(self):
        return Const(-1) * self

    def __pos__(self):
        return self

Создайте классы для двух видов выражений: `Const`, представляющий константу, и` Var`, представляющий переменную. Чтобы упростить использование, вместо обращения к конструкторам этих классов, будем использовать их однобуквенные сокращённые обозначения.

**Пример использования:**
```python
V = Var
C = Const

C(5)()
5
C(5).d(V("x"))()
0
V("x")(x=5)
5
V("x").d(V("y"))(x=5)
0
V("x").d(V("x"))(x=5)
1
```

In [10]:
class Const(Expr):
    def __init__(self, const):
        self.const = const

    def __call__(self, **context):
        return self.const

    def d(self, var):
        return Const(0)


class Var(Expr):
    def __init__(self, var):
        self.var = var

    def __call__(self, **context):
        if self.var in context:
            return context[self.var]
        else:
            raise ValueError(f"Значение переменной {self.var} не было передано")

    def d(self, var):
        return Const(int(self.var == var.var))


V = Var
C = Const

In [11]:
C(5)()

5

In [12]:
C(5).d(V("x"))()

0

In [18]:
V("x")(x=5)

5

In [21]:
V("x").d(V("y"))(x=5)

0

In [23]:
V("x").d(V("x"))(x=5)

1

## Бинарные операции

Создайте классы для бинарных операций: `Sum`, `Product` и `Fraction`. Поскольку бинарные операции определяются двумя операндами, их конструктор будет одинаковым для всех этих классов. Поэтому его можно вынести в отдельный базовый класс, чтобы избежать дублирования кода.

In [24]:
class BinOp(Expr):
    def __init__(self, expr1, expr2):
        self.expr1, self.expr2 = expr1, expr2

Реализуйте `Sum` для суммирования, `Product` для умножения и `Fraction` для деления.

**Пример использования:**

```python
Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)
7.0
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)
3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)
-3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)
-1.25
```

In [25]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, var):
        u = self.expr1
        du = u.d(var)
        v = self.expr2
        dv = v.d(var)
        return Sum(du, dv)


class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)

    def d(self, var):
        u = self.expr1
        du = u.d(var)
        v = self.expr2
        dv = v.d(var)
        vdu = Product(v, du)
        udv = Product(u, dv)
        return Sum(vdu, udv)


class Fraction(BinOp):
    def __call__(self, **context):
        if self.expr2(**context) == 0:
            raise ZeroDivisionError("Знаменатель не должен равняться 0")
        return self.expr1(**context) / self.expr2(**context)

    def d(self, var):
        u = self.expr1
        du = u.d(var)
        v = self.expr2
        dv = v.d(var)
        vdu = Product(v, du)
        udv = Product(u, dv)
        mudv = Product(Const(-1), udv)
        vdumudv = Sum(vdu, mudv)
        vp2 = Product(v, v)
        return Fraction(vdumudv, vp2)

In [26]:
Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)

7.0

In [27]:
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)

3.5

In [29]:
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)

-3.5

In [30]:
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)

-1.25

## Перегрузка операторов

Добавьте перегрузку операторов в базовых класс `Expr`. Обратите что в классах мы можем тоже заменить на использование операторов.
```python  
-e         e.__neg__()
+e         e.__pos__()
e1 + e2    e1.__add__(e2)
e1 - e2    e1.__sub__(e2)
e1 * e2    e1.__mul__(e2)
e1 / e2    e1.__truediv__(e2)
```

**Пример использования:**

```python
(V("x") * V("x") / V("y"))(x=5, y=2.5)
10.0
```

In [33]:
(V("x") * V("x") / V("y"))(x=5, y=2.5)

10.0

## Метод Ньютона-Рафсона

Напишите функцию `newton_raphson`, которая принимает дифференцируемую функцию  $f$  от переменной  $x$ , начальное приближение  $x_0$ , и положительное число  $\epsilon$ , задающее точность вычислений. Функция должна возвращать значение  $x$ , при котором  $f(x)$  становится равным нулю. Метод Ньютона-Рафсона выполняет итеративный поиск корня функции  $f(x)$ , начиная с начального значения  $x_0$ , и использует правило  
$$x_{n+1} = x_n - \frac{f(x_n)}{f{\prime}(x_n)}$$  
для обновления  $x$  на каждом шаге. Итерации продолжаются до тех пор, пока условие остановки  $|x_{n+1} - x_n| \leq \epsilon$  не будет выполнено.

**Пример использования:**

```python
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)
(1.000000000001132, -2.490496697760136e-11)
```

In [31]:
def newton_raphson(expr, x0, eps=1e-4):
    x = V("x")
    g = x - expr / expr.d(x)
    xn = x0
    xn1 = xn + 2 * eps
    while abs(xn1 - xn) > eps:
        xn = xn1
        xn1 = g(x=xn1)
    return xn1

In [32]:
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)

(1.000000000001132, -2.490496697760136e-11)